# Keras 快速入门

Keras 是一个用于构建和训练深度学习模型的高阶 API。它可用于快速设计原型、高级研究和生产。

keras的3个优点： 方便用户使用、模块化和可组合、易于扩展

## 导入Keras

tensorflow2推荐使用keras构建网络，常见的神经网络都包含在keras.layer中(最新的tf.keras的版本可能和keras不同)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

1.14.0
2.2.4-tf


## 构建简单模型

### 模型堆叠

最常见的模型类型是层的堆叠：tf.keras.Sequential 模型

In [3]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

W0810 20:41:26.426313  6468 deprecation.py:506] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### 网络配置

tf.keras.layers中网络配置：

activation：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数。

kernel_initializer 和 bias_initializer：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器。

kernel_regularizer 和 bias_regularizer：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。

In [4]:
layers.Dense(32, activation='sigmoid')
layers.Dense(32, activation=tf.sigmoid)
layers.Dense(32, kernel_initializer='orthogonal')
layers.Dense(32, kernel_initializer=tf.keras.initializers.glorot_normal)
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l1(0.01))

## 训练和评估

### 设置训练流程

构建好模型后，通过调用 compile 方法配置该模型的学习流程：

In [5]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

### 输入 numpy 数据

In [6]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 1s 626us/sample - loss: 12.5915 - categorical_accuracy: 0.1080 - val_loss: 12.9621 - val_categorical_accuracy: 0.1300
Epoch 2/10
1000/1000 [==============================] - 0s 21us/sample - loss: 13.0380 - categorical_accuracy: 0.1230 - val_loss: 13.8799 - val_categorical_accuracy: 0.1200
Epoch 3/10
1000/1000 [==============================] - 0s 21us/sample - loss: 14.3821 - categorical_accuracy: 0.0850 - val_loss: 16.1149 - val_categorical_accuracy: 0.1050
Epoch 4/10
1000/1000 [==============================] - 0s 21us/sample - loss: 17.4926 - categorical_accuracy: 0.0890 - val_loss: 20.7250 - val_categorical_accuracy: 0.1150
Epoch 5/10
1000/1000 [==============================] - 0s 20us/sample - loss: 22.8636 - categorical_accuracy: 0.0870 - val_loss: 27.0735 - val_categorical_accuracy: 0.1150
Epoch 6/10
1000/1000 [==============================] - 0s 20us/sample - loss: 28.7496 

### tf.data 输入数据

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

W0810 20:55:31.986442  6468 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 109.7410 - categorical_accuracy: 0.0938 - val_loss: 155.0785 - val_categorical_accuracy: 0.0625
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 184.7956 - categorical_accuracy: 0.0940 - val_loss: 250.6502 - val_categorical_accuracy: 0.0938
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 290.1126 - categorical_accuracy: 0.0919 - val_loss: 376.1395 - val_categorical_accuracy: 0.1146
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 410.4020 - categorical_accuracy: 0.0951 - val_loss: 517.3589 - val_categorical_accuracy: 0.1250
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 552.3217 - categorical_accuracy: 0.0908 - val_loss: 675.9874 - val_categorical_accuracy: 0.1250
Epoch 6/10
30/30 [==============================] - 0s 2ms/step - loss: 704.1227 - categorical_accuracy: 0.0833 - val_loss: 848.0327 - val_categorical_accuracy:

### 评估和预测

In [8]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)
# predict
result = model.predict(test_x, batch_size=32)
print(result)

30/30 [==============================] - 0s 2ms/step - loss: 1370.2131 - categorical_accuracy: 0.0938
[[0.         0.         0.00567246 ... 0.28867733 0.21939978 0.        ]
 [0.         0.         0.00341834 ... 0.2752905  0.25163588 0.        ]
 [0.         0.         0.00230477 ... 0.20787466 0.35291672 0.        ]
 ...
 [0.         0.         0.00470895 ... 0.22889282 0.20759678 0.        ]
 [0.         0.         0.00225743 ... 0.18556562 0.37121    0.        ]
 [0.         0.         0.00754409 ... 0.28670672 0.31396455 0.        ]]


## 构建高级模型

### 函数式 api

tf.keras.Sequential 模型是层的简单堆叠，无法表示任意模型。使用 Keras 函数式 API 可以构建复杂的模型拓扑，例如：

多输入模型，

多输出模型，

具有共享层的模型（同一层被调用多次），

具有非序列数据流的模型（例如，残差连接）。

**使用函数式 API 构建的模型具有以下特征：**

层实例可调用并返回张量。 输入张量和输出张量用于定义 tf.keras.Model 实例。 此模型的训练方式和 Sequential 模型一样。

In [9]:
input_x = tf.keras.Input(shape=(72,))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 100us/sample - loss: 12.6879 - acc: 0.0940
Epoch 2/5
1000/1000 [==============================] - 0s 52us/sample - loss: 18.5924 - acc: 0.1070
Epoch 3/5
1000/1000 [==============================] - 0s 52us/sample - loss: 33.6198 - acc: 0.1060
Epoch 4/5
1000/1000 [==============================] - 0s 51us/sample - loss: 62.7332 - acc: 0.1050
Epoch 5/5
1000/1000 [==============================] - 0s 49us/sample - loss: 107.9833 - acc: 0.1020


### 模型子类化

通过对 tf.keras.Model 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。在 init 方法中创建层并将它们设置为类实例的属性。在 call 方法中定义前向传播

In [10]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 140us/sample - loss: 14.1145 - acc: 0.0940
Epoch 2/5
1000/1000 [==============================] - 0s 89us/sample - loss: 17.8594 - acc: 0.1130
Epoch 3/5
1000/1000 [==============================] - 0s 92us/sample - loss: 20.7122 - acc: 0.1120
Epoch 4/5
1000/1000 [==============================] - 0s 91us/sample - loss: 22.3641 - acc: 0.1010
Epoch 5/5
1000/1000 [==============================] - 0s 91us/sample - loss: 22.7968 - acc: 0.1030


### 自定义层

通过对 tf.keras.layers.Layer 进行子类化并实现以下方法来创建自定义层：

build：创建层的权重。使用 add_weight 方法添加权重。

call：定义前向传播。

compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。 或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。

In [11]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = tf.keras.Sequential([MyLayer(10), layers.Activation('softmax')])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

W0810 21:09:33.791243  6468 deprecation.py:506] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/5
1000/1000 [==============================] - 0s 109us/sample - loss: 11.5615 - acc: 0.1020
Epoch 2/5
1000/1000 [==============================] - 0s 70us/sample - loss: 11.5611 - acc: 0.0990
Epoch 3/5
1000/1000 [==============================] - 0s 73us/sample - loss: 11.5596 - acc: 0.0980
Epoch 4/5
1000/1000 [==============================] - 0s 72us/sample - loss: 11.5598 - acc: 0.0990
Epoch 5/5
1000/1000 [==============================] - 0s 73us/sample - loss: 11.5576 - acc: 0.0990


### 回调

In [16]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]

model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks, validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 0s 147us/sample - loss: 11.5581 - acc: 0.1010 - val_loss: 11.6532 - val_acc: 0.1150
Epoch 2/5
1000/1000 [==============================] - 0s 84us/sample - loss: 11.5560 - acc: 0.0990 - val_loss: 11.6512 - val_acc: 0.1150
Epoch 3/5
1000/1000 [==============================] - 0s 87us/sample - loss: 11.5548 - acc: 0.0990 - val_loss: 11.6531 - val_acc: 0.1150
Epoch 4/5
1000/1000 [==============================] - 0s 88us/sample - loss: 11.5536 - acc: 0.0980 - val_loss: 11.6519 - val_acc: 0.1150


## 保存和恢复

### 权重保存

In [23]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

model.save_weights('./ch1/weights/model')
model.load_weights('./ch1/weights/model')
model.save_weights('./ch1/model.h5')
model.load_weights('./ch1/model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 146us/sample - loss: 12.9275 - acc: 0.0930
Epoch 2/5
1000/1000 [==============================] - 0s 90us/sample - loss: 15.0984 - acc: 0.1010
Epoch 3/5
1000/1000 [==============================] - 0s 87us/sample - loss: 17.1111 - acc: 0.0930
Epoch 4/5
1000/1000 [==============================] - 0s 89us/sample - loss: 19.2364 - acc: 0.0970
Epoch 5/5
1000/1000 [==============================] - 0s 90us/sample - loss: 19.7631 - acc: 0.0970


### 保存网络结构

In [21]:
# 序列化成json
import json
import pprint

json_str = model.to_json()
# 使用 pprint 美观打印
print('序列化为 json:')
pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)

print('----------------------------------------------')

# 保持为yaml格式  
# 需要提前安装pyyaml
yaml_str = model.to_yaml()
print('序列化为 yaml:')
print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

序列化为 json:
{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'build_input_shape': [None, 72],
            'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'kernel_regulari

### 保存整个模型

In [24]:
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(72,)),
  layers.Dense(10, activation='softmax')])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=32, epochs=5)

model.save('./ch1/all_model.h5')
model = tf.keras.models.load_model('./ch1/all_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 111us/sample - loss: 11.5430 - acc: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 50us/sample - loss: 11.5847 - acc: 0.1090
Epoch 3/5
1000/1000 [==============================] - 0s 49us/sample - loss: 11.6595 - acc: 0.0960
Epoch 4/5
1000/1000 [==============================] - 0s 48us/sample - loss: 11.6954 - acc: 0.0960
Epoch 5/5
1000/1000 [==============================] - 0s 48us/sample - loss: 11.7049 - acc: 0.0960


## 将 Keras 用于 Estimator

Estimator API 用于针对分布式环境训练模型。它适用于一些行业使用场景，例如用大型数据集进行分布式训练并导出模型以用于生产

In [26]:
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

W0810 21:42:23.990992  6468 estimator.py:1811] Using temporary folder as model directory: C:\Users\44310\AppData\Local\Temp\tmp64e3tbtd


只要用 tf.keras.estimator.model_to_estimator 方法将 tf.keras.Model 转换为 tf.estimator.Estimator 对象, tf.keras.Model 就可以使用 tf.estimator API 训练